# 전원MinMax

In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

In [2]:
# origin=read_excel('행정동병합테스트22.xlsx')
origin=read_excel('행정동병합테스트22(개발제한행정동제외).xlsx')
origin.head()

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,626,110,956.51,3,1.000000,0,16,0,5,1728.859060
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,1814,311,0.00,3,0.000000,10,34,0,5,4130.837004
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,3495,491,1.14,3,0.001192,10,54,0,5,1990.755355


In [3]:
origin.isna().sum()

Unnamed: 0       0
시도명              0
시군구코드            0
시군구명             0
행정동코드            0
행정동코드_수정         0
읍면동명             0
생성일자             0
말소일자             0
지하철역_수           0
의료기관_수           0
사회복지시설_수         0
계(명)             0
면적 (k㎡)          0
구성비 (%)          0
행정동별_공원수         0
공공체육시설_수         0
문화시설_수           0
학교_수             0
사업체_수            0
개소 (공영+민영)       0
면수 (공영+민영)       0
하천및한강여부          0
아동인구(0~9세)       0
노인인구(65세이상)      0
장애인_인구           0
세대당공원면적          0
보육시설수            0
세대당공원면적minmax    0
수변점수             0
행정동별버스정류장수       0
행정동별유통업체수        0
공공자전거대여소수        0
인구밀도             0
dtype: int64

In [4]:
origin1=origin.fillna(0)

In [5]:
scaler=MinMaxScaler()

In [6]:
origin1.columns

Index(['Unnamed: 0', '시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명',
       '생성일자', '말소일자', '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)',
       '구성비 (%)', '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수',
       '개소 (공영+민영)', '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)',
       '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수',
       '행정동별유통업체수', '공공자전거대여소수', '인구밀도'],
      dtype='object')

In [7]:
df = origin1.copy()
df['의료기관minmax']=scaler.fit_transform(df[['의료기관_수']])
df['사회복지minmax']=scaler.fit_transform(df[['사회복지시설_수']])
df['보육시설minmax']=scaler.fit_transform(df[['보육시설수']])
df['노인인구minmax']=scaler.fit_transform(df[['노인인구(65세이상)']])
df['아동인구minmax']=scaler.fit_transform(df[['아동인구(0~9세)']])
df['장애인minmax']=scaler.fit_transform(df[['장애인_인구']])
df['면적minmax']=scaler.fit_transform(df[['면적 (k㎡)']])
df.head()

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,행정동별유통업체수,공공자전거대여소수,인구밀도,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,0,3,4552.529183,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.270833
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,0,12,7611.382114,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.115741
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,0,5,1728.859060,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.145833
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,0,5,4130.837004,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.236111
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,0,5,1990.755355,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,1.000000


In [8]:
df.columns

Index(['Unnamed: 0', '시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명',
       '생성일자', '말소일자', '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)',
       '구성비 (%)', '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수',
       '개소 (공영+민영)', '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)',
       '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수',
       '행정동별유통업체수', '공공자전거대여소수', '인구밀도', '의료기관minmax', '사회복지minmax',
       '보육시설minmax', '노인인구minmax', '아동인구minmax', '장애인minmax', '면적minmax'],
      dtype='object')

In [9]:
# 면적대비_거주하는_사회적_약자_수 파생지수(1) 생성
df['면적대비_거주하는_사회적_약자_수'] = (df['노인인구minmax'] + df['아동인구minmax'] + df['장애인minmax']) / df['면적minmax']
df.head()

# df['사회적약자지수'] = (df['의료기관minmax'] + df['사회복지minmax'] + df['보육시설minmax']) / (df['노인인구minmax'] + df['아동인구minmax'] + df['장애인minmax'])
# df.head()

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,공공자전거대여소수,인구밀도,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax,면적대비_거주하는_사회적_약자_수
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,3,4552.529183,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.270833,1.692327
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,12,7611.382114,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.115741,3.050415
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,5,1728.859060,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.145833,0.797650
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,5,4130.837004,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.236111,1.482259
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,5,1990.755355,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,1.000000,0.691319


In [10]:
df['면적대비_거주하는_사회적_약자_수'].value_counts()

# '면적대비_거주하는_사회적_약자_수' 열에서 'inf' 값을 0으로 변경
df['면적대비_거주하는_사회적_약자_수'].replace([np.inf, -np.inf], 0, inplace=True)

df['면적대비_거주하는_사회적_약자_수minmax']=scaler.fit_transform(df[['면적대비_거주하는_사회적_약자_수']])
df

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,인구밀도,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,4552.529183,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.270833,1.692327,0.013697
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,7611.382114,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.115741,3.050415,0.024690
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,1728.859060,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.145833,0.797650,0.006456
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,4130.837004,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.236111,1.482259,0.011997
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,1990.755355,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,1.000000,0.691319,0.005595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0,3,...,34780.597015,0.096875,0.222222,0.272727,0.510127,0.137894,0.294021,0.050926,18.498275,0.149723
402,422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0,1,...,32228.169014,0.125000,0.222222,0.333333,0.467674,0.157417,0.249589,0.055556,15.744233,0.127432
403,423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0,1,...,28038.509317,0.160938,0.407407,0.515152,0.968875,0.316833,0.544432,0.159722,11.458272,0.092742
404,424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0,1,...,76.086957,0.001563,0.000000,0.000000,0.000000,0.000154,0.000000,0.079861,0.001925,0.000016


In [11]:
# 면적대비_거주하는_사회적_약자_수 파생지수(1) 생성
df['사회적약자_지원_인프라_밀집도'] = (df['의료기관minmax'] + df['사회복지minmax'] + df['보육시설minmax']) / df['면적대비_거주하는_사회적_약자_수minmax']
df.head()

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax,사회적약자_지원_인프라_밀집도
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.270833,1.692327,0.013697,23.623643
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.115741,3.050415,0.024690,6.246513
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.145833,0.797650,0.006456,16.743353
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.236111,1.482259,0.011997,20.446984
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,1.000000,0.691319,0.005595,93.246053


In [12]:
df['사회적약자_지원_인프라_밀집도'].value_counts().sort_index()

사회적약자_지원_인프라_밀집도
0.059172      1
0.121651      1
0.186858      1
0.411431      1
0.555827      1
             ..
100.287255    1
131.038687    1
140.547466    1
141.560361    1
inf           2
Name: count, Length: 405, dtype: int64

In [13]:
df['사회적약자_지원_인프라_밀집도'].isna().sum()

0

In [14]:
df['사회적약자_지원_인프라_밀집도'].replace([np.inf, -np.inf], 0, inplace=True)
df

,Unnamed: 0,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,...,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax,사회적약자_지원_인프라_밀집도
0,0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,...,0.017188,0.185185,0.121212,0.245223,0.094081,0.119035,0.270833,1.692327,0.013697,23.623643
1,1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,...,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.115741,3.050415,0.024690,6.246513
2,2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,...,0.017188,0.000000,0.090909,0.070374,0.016603,0.029347,0.145833,0.797650,0.006456,16.743353
3,3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,...,0.006250,0.148148,0.090909,0.206317,0.059185,0.084476,0.236111,1.482259,0.011997,20.446984
4,4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,...,0.023438,0.407407,0.090909,0.398673,0.158801,0.133845,1.000000,0.691319,0.005595,93.246053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0,3,...,0.096875,0.222222,0.272727,0.510127,0.137894,0.294021,0.050926,18.498275,0.149723,3.952799
402,422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0,1,...,0.125000,0.222222,0.333333,0.467674,0.157417,0.249589,0.055556,15.744233,0.127432,5.340540
403,423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0,1,...,0.160938,0.407407,0.515152,0.968875,0.316833,0.544432,0.159722,11.458272,0.092742,11.682921
404,424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0,1,...,0.001563,0.000000,0.000000,0.000000,0.000154,0.000000,0.079861,0.001925,0.000016,100.287255


In [15]:
df['사회적약자_지원_인프라_밀집도'].unique()

array([2.36236431e+01, 6.24651295e+00, 1.67433532e+01, 2.04469835e+01,
       9.32460529e+01, 7.45657260e-01, 1.02948514e+00, 3.56299114e+00,
       6.39085910e+01, 9.76306388e+00, 9.76515118e+00, 5.82718445e+00,
       7.32244871e-01, 1.21651346e-01, 0.00000000e+00, 7.55028187e-01,
       3.47428141e+01, 1.38840926e+01, 4.93081022e+01, 1.59984788e+01,
       6.73866747e+00, 3.88335466e+00, 1.04299624e+01, 1.51415816e+00,
       2.22113172e+00, 1.79149123e+00, 5.55826899e-01, 1.00383538e+00,
       1.86858165e-01, 6.21418879e-01, 3.34826846e+00, 7.60586299e+00,
       6.50063954e+00, 1.92313483e+01, 5.36410080e+00, 3.95388571e+00,
       4.94457103e+00, 4.23789144e+00, 4.11430764e-01, 2.18050521e+01,
       1.55359542e+01, 6.33447402e+00, 5.20032741e+00, 2.80157527e+00,
       2.28496778e+01, 1.72635361e+01, 2.41005010e+00, 1.45955162e+00,
       6.77055042e+00, 9.44911155e+00, 7.87935499e+00, 3.92153677e+00,
       1.29028821e+00, 1.93504430e+00, 1.98580349e+00, 3.61857058e+00,
      

In [16]:
df.columns

Index(['Unnamed: 0', '시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명',
       '생성일자', '말소일자', '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)',
       '구성비 (%)', '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수',
       '개소 (공영+민영)', '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)',
       '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수',
       '행정동별유통업체수', '공공자전거대여소수', '인구밀도', '의료기관minmax', '사회복지minmax',
       '보육시설minmax', '노인인구minmax', '아동인구minmax', '장애인minmax', '면적minmax',
       '면적대비_거주하는_사회적_약자_수', '면적대비_거주하는_사회적_약자_수minmax', '사회적약자_지원_인프라_밀집도'],
      dtype='object')

In [17]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','의료기관수','사회복지시설수','보육시설','노인인구','장애인인구','아동인구','면적 (k㎡)','의료기관minmax','사회복지minmax','보육시설minmax','노인인구minmax','장애인minmax','아동인구minmax','면적대비_거주하는_사회적_약자_수','면적대비_거주하는_사회적_약자_수minmax','사회적약자_지원_인프라_밀집도'])
df1

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,장애인minmax,아동인구minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax,사회적약자_지원_인프라_밀집도
0,11110,종로구,1111051500,청운효자동,2.57,0.017188,0.185185,0.121212,0.245223,0.119035,0.094081,1.692327,0.013697,23.623643
1,11110,종로구,1111053000,사직동,1.23,0.117188,0.037037,0.000000,0.205630,0.072408,0.075019,3.050415,0.024690,6.246513
2,11110,종로구,1111054000,삼청동,1.49,0.017188,0.000000,0.090909,0.070374,0.029347,0.016603,0.797650,0.006456,16.743353
3,11110,종로구,1111055000,부암동,2.27,0.006250,0.148148,0.090909,0.206317,0.084476,0.059185,1.482259,0.011997,20.446984
4,11110,종로구,1111056000,평창동,8.87,0.023438,0.407407,0.090909,0.398673,0.133845,0.158801,0.691319,0.005595,93.246053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,11740,강동구,1174065000,성내제2동,0.67,0.096875,0.222222,0.272727,0.510127,0.294021,0.137894,18.498275,0.149723,3.952799
402,11740,강동구,1174066000,성내제3동,0.71,0.125000,0.222222,0.333333,0.467674,0.249589,0.157417,15.744233,0.127432,5.340540
403,11740,강동구,1174068500,길동,1.61,0.160938,0.407407,0.515152,0.968875,0.544432,0.316833,11.458272,0.092742,11.682921
404,11740,강동구,1174069000,둔촌제1동,0.92,0.001563,0.000000,0.000000,0.000000,0.000000,0.000154,0.001925,0.000016,100.287255


In [18]:
df1['사회적약자_지원_인프라_밀집도minmax']=scaler.fit_transform(df1[['사회적약자_지원_인프라_밀집도']])
df1

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,장애인minmax,아동인구minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax,사회적약자_지원_인프라_밀집도,사회적약자_지원_인프라_밀집도minmax
0,11110,종로구,1111051500,청운효자동,2.57,0.017188,0.185185,0.121212,0.245223,0.119035,0.094081,1.692327,0.013697,23.623643,0.166880
1,11110,종로구,1111053000,사직동,1.23,0.117188,0.037037,0.000000,0.205630,0.072408,0.075019,3.050415,0.024690,6.246513,0.044126
2,11110,종로구,1111054000,삼청동,1.49,0.017188,0.000000,0.090909,0.070374,0.029347,0.016603,0.797650,0.006456,16.743353,0.118277
3,11110,종로구,1111055000,부암동,2.27,0.006250,0.148148,0.090909,0.206317,0.084476,0.059185,1.482259,0.011997,20.446984,0.144440
4,11110,종로구,1111056000,평창동,8.87,0.023438,0.407407,0.090909,0.398673,0.133845,0.158801,0.691319,0.005595,93.246053,0.658702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,11740,강동구,1174065000,성내제2동,0.67,0.096875,0.222222,0.272727,0.510127,0.294021,0.137894,18.498275,0.149723,3.952799,0.027923
402,11740,강동구,1174066000,성내제3동,0.71,0.125000,0.222222,0.333333,0.467674,0.249589,0.157417,15.744233,0.127432,5.340540,0.037726
403,11740,강동구,1174068500,길동,1.61,0.160938,0.407407,0.515152,0.968875,0.544432,0.316833,11.458272,0.092742,11.682921,0.082530
404,11740,강동구,1174069000,둔촌제1동,0.92,0.001563,0.000000,0.000000,0.000000,0.000000,0.000154,0.001925,0.000016,100.287255,0.708442


In [19]:
df1.to_excel('사회적약자지수1.xlsx',index=False)